Image Classification using Convolutional Neural Network

In [0]:
!pip install pydrive

    100% |████████████████████████████████| 993kB 25.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import AveragePooling2D,Dropout,Flatten
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator

#--------------linear algebra, data processing----------#
import numpy as np 
import pandas as pd
import glob,string
import codecs
from tqdm import tqdm

#--------------Google authentication--------------------#
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
import tensorflow as tf
from oauth2client.client import GoogleCredentials

Using TensorFlow backend.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
zip_file = drive.CreateFile({'id': '1Kc-NFM_mfvnrx31rcpsnrMWarEb2Gnmc'})
zip_file.GetContentFile('3D_object_dataset.zip')

In [0]:
!unzip '3D_object_dataset.zip'

In [0]:
training_directory = '3D_object_dataset/Training'
validation_directory = '3D_object_dataset/validation'
testing_directory = '3D_object_dataset/Testing'

In [0]:
batch_size = 15
epochs = 100

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=20,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        training_directory,
        target_size=(128,128),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

validation_generator = val_datagen.flow_from_directory(
        validation_directory,
        target_size=(128,128),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle = True)


Found 1605 images belonging to 4 classes.
Found 264 images belonging to 4 classes.


In [0]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', input_shape=(128,128,1)))
model.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
model.add(AveragePooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.25))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.3))

model.add(Conv2D(512, kernel_size=(2, 2), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(1024, kernel_size=(2, 2), activation='relu'))
model.add(Conv2D(512, kernel_size=(2, 2), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_102 (Conv2D)          (None, 124, 124, 64)      1664      
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 120, 120, 64)      102464    
_________________________________________________________________
average_pooling2d_55 (Averag (None, 40, 40, 64)        0         
_________________________________________________________________
dropout_103 (Dropout)        (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 38, 38, 128)       73856     
_________________________________________________________________
average_pooling2d_56 (Averag (None, 19, 19, 128)       0         
_________________________________________________________________
dropout_104 (Dropout)        (None, 19, 19, 128)       0         
__________

In [0]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
patience =30

#log_file_path = base_path + dataset_name + '_emotion_training.log'
#csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
trained_models_path = 'trained_model_2'
model_names = trained_models_path + '.{epoch:02d}.{val_acc:02f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

model.fit_generator(generator=train_generator,
	                    steps_per_epoch=66,
	                    validation_data=validation_generator,
	                    validation_steps=16,
	                    epochs=100,
                      callbacks= callbacks)

Epoch 1/100
66/66 [==============================] - 18s 278ms/step - loss: 1.4426 - acc: 0.2808 - val_loss: 1.3929 - val_acc: 0.2436

Epoch 00001: val_loss improved from inf to 1.39290, saving model to trained_model_2.01.0.243590.hdf5
Epoch 2/100
66/66 [==============================] - 12s 187ms/step - loss: 1.3767 - acc: 0.3192 - val_loss: 1.4199 - val_acc: 0.2265

Epoch 00002: val_loss did not improve from 1.39290
Epoch 3/100
66/66 [==============================] - 14s 208ms/step - loss: 1.3870 - acc: 0.2838 - val_loss: 1.3993 - val_acc: 0.2393

Epoch 00003: val_loss did not improve from 1.39290
Epoch 4/100
66/66 [==============================] - 15s 228ms/step - loss: 1.3818 - acc: 0.2990 - val_loss: 1.3972 - val_acc: 0.2222

Epoch 00004: val_loss did not improve from 1.39290
Epoch 5/100
66/66 [==============================] - 14s 218ms/step - loss: 1.3800 - acc: 0.3071 - val_loss: 1.3956 - val_acc: 0.2417

Epoch 00005: val_loss did not improve from 1.39290
Epoch 6/100
66/66 [=

In [0]:
model_json = model.to_json()
with open("model_conv.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_conv.h5")